## Процесс обработки данных

In [30]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic
import os
import pickle


df = pd.read_csv("/home/zemfira/Фаза 0/Data-analysis-of-the-service-for-the-sale-and-rental-of-residential-real-estate/_data.csv")
pd.set_option('display.max_columns', None)
df.head(5)

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615


#### Исследование проводится по Москве, поэтому выделяем только этот город в столбце **Адрес**

In [31]:
df = df[df['Адрес'].str.contains("Москва")] 

####  Добавили новые признаки на основе старых:
##### **Цена** была разбита на три показателя - Rent, Deposit, Result

In [32]:
def extract_rent(price_str):
    match = re.search(r'(\d+(?:\.\d+)?)\s*руб', price_str)
    if match:
        return float(match.group(1))
    return None

def extract_deposit(price_str):
    match = re.search(r'Залог\s*-\s*(\d+(?:\.\d+)?)\s*руб', price_str, re.IGNORECASE)
    if match:
        return float(match.group(1))
    return 0.0  

df['Rent'] = df['Цена'].apply(extract_rent)
df['Deposit'] = df['Цена'].apply(extract_deposit)
df['Result'] = df['Rent'] + df['Deposit']

df['Rent'] = df['Rent'].fillna(0)
df['Deposit'] = df['Deposit'].fillna(0)
df['Result'] = df['Result'].fillna(0)

df['Rent'] = df['Rent'].astype(float)
df['Deposit'] = df['Deposit'].astype(float)
df['Result'] = df['Result'].astype(float)

##### **Лифт** был разбит на два показателя - Passenger_elevator, Freight_elevator

In [33]:
def extract_pass_lifts(lift_str): #пассажир
    if pd.isna(lift_str):
        return 0
    match = re.search(r'Пасс\s*\(?\s*(\d+)\s*\)?', lift_str, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return 0 

def extract_cargo_lifts(lift_str): #грузовые
    if pd.isna(lift_str):
        return 0
    match = re.search(r'Груз\s*\(?\s*(\d+)\s*\)?', lift_str, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return 0

df['Passenger_elevator'] = df['Лифт'].apply(extract_pass_lifts)
df['Freight_elevator'] = df['Лифт'].apply(extract_cargo_lifts)

df['Passenger_elevator'] = df['Passenger_elevator'].fillna(0).astype(int)
df['Freight_elevator'] = df['Freight_elevator'].fillna(0).astype(int)

In [34]:
def sum_room_areas(area_str):
    if pd.isna(area_str):
        return 0
    numbers = re.findall(r'\d+', area_str)
    return sum(map(int, numbers)) if numbers else 0

df['The_total_area_of_the_rooms_m2'] = df['Площадь комнат, м2'].apply(sum_room_areas)

TypeError: 'numpy.int64' object is not callable

##### признак **Дом** был разбит на два показателя - Number_of_floors, Type_of_building_of_the_house, Current_floor, Total_number_of_floors

In [35]:
df[['Number_of_floors', 'Type_of_building_of_the_house']] = df['Дом'].str.split(',', expand=True)
df['Number_of_floors'] = df['Number_of_floors'].str.strip()
df['Type_of_building_of_the_house'] = df['Type_of_building_of_the_house'].str.strip()
df['Type_of_building_of_the_house'] = df['Type_of_building_of_the_house'].fillna('Не указано')
df[['Current_floor', 'Total_number_of_floors']] = df['Number_of_floors'].str.split('/', expand=True)
df['Current_floor'] = pd.to_numeric(df['Current_floor'], errors='coerce')
df['Total_number_of_floors'] = pd.to_numeric(df['Total_number_of_floors'], errors='coerce')

##### Заполнили пропуски в показателе **Количество комнат** с помощью среднего значения по этому столбцу

In [36]:
df['Количество комнат'] = df['Количество комнат'].str.extract('(\d+)').astype(float)
среднее_количество = df['Количество комнат'].mean()
df['Количество комнат'] = df['Количество комнат'].fillna(среднее_количество)
df['Количество комнат'] = df['Количество комнат'].astype(int)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_26000/2803180896.py:1: SyntaxWarning: invalid escape sequence '\d'
  df['Количество комнат'] = df['Количество комнат'].str.extract('(\d+)').astype(float)


##### Признак **Площадь** разделили на **Общую площадь**, **Жилая площадь**, **Площадь кухни**

In [37]:
df[['Total_area', 'Living_area', 'Kitchen']] = df['Площадь, м2'].str.split('/', expand=True)

df['Total_area'] = df['Total_area'].fillna('0')
df['Living_area'] = df['Living_area'].fillna('0')
df['Kitchen'] = df['Kitchen'].fillna('0')

df['Total_area'] = pd.to_numeric(df['Total_area'].str.replace(',', '.'), errors='coerce').fillna(0)
df['Living_area'] = pd.to_numeric(df['Living_area'].str.replace(',', '.'), errors='coerce').fillna(0)
df['Kitchen'] = pd.to_numeric(df['Kitchen'].str.replace(',', '.'), errors='coerce').fillna(0)

##### Все колонки привели к требуемому типу

In [38]:
rename_dict = {'ID  объявления': 'ID', 
               'Количество комнат': 'Number_of_rooms', 
               'Тип': 'Type', 
               'Метро': 'Metro', 
               'Адрес': 'Address', 
               'Площадь, м2': 'Area_m2', 
               'Дом': 'Home', 
               'Парковка': 'Parking', 
               'Цена': 'Price', 
               'Телефоны': 'Phone', 
               'Описание': 'Description', 
               'Ремонт': 'Renovation', 
               'Площадь комнат, м2': 'Room_area_m2', 
               'Балкон': 'Balcony', 
               'Окна': 'Windows', 
               'Санузел': 'Bathroom', 
               'Можно с детьми/животными': 'With_children/animals', 
               'Дополнительно': 'Additionally', 
               'Название ЖК': 'Name_of_the_RC', 
               'Серия дома': 'House_Series', 
               'Высота потолков, м': 'Ceiling_height_m', 
               'Лифт': 'Elevator', 
               'Мусоропровод': 'Garbage_chute', 
               'Ссылка на объявление': 'Link_to_the_ad'
              }

df = df.rename(columns=rename_dict)

##### Перемещение колонок в начало таблицы

In [39]:
replacements = {
    'Area_m2': ['Total_area', 'Living_area', 'Kitchen'],
    'Price': ['Rent', 'Deposit', 'Result'],
    'Elevator': ['Passenger_elevator', 'Freight_elevator'],
    'Room_area_m2': ['The_total_area_of_the_rooms_m2'],
    'Home': ['Type_of_building_of_the_house', 'Current_floor', 'Total_number_of_floors', 'Number_of_floors'] 
}

replacement_cols = set(col for cols in replacements.values() for col in cols)


new_order = []
for col in df.columns:
    if col in replacements:

        new_order.extend(replacements[col])
    else:
        if col not in replacement_cols:
            new_order.append(col)

df = df[new_order]

KeyError: "['The_total_area_of_the_rooms_m2'] not in index"

##### Удаление колонок с наибольшими количествами пропусков

In [40]:
columns_to_drop = ['House_Series', 'Name_of_the_RC', 'Parking']

existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
missing_columns = [col for col in columns_to_drop if col not in df.columns]

if existing_columns_to_drop:
    df = df.drop(columns=existing_columns_to_drop)

df.head(3)

,Unnamed: 0,ID,Number_of_rooms,Type,Metro,Address,Area_m2,Home,Price,Phone,Description,Renovation,Room_area_m2,Balcony,Windows,Bathroom,With_children/animals,Additionally,Ceiling_height_m,Elevator,Garbage_chute,Link_to_the_ad,Rent,Deposit,Result,Passenger_elevator,Freight_elevator,Number_of_floors,Type_of_building_of_the_house,Current_floor,Total_number_of_floors,Total_area,Living_area,Kitchen
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный","500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,500000.0,500000.0,1000000.0,4,1,5/16,Монолитный,5,16,200.0,20.0,0.0
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный","500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,500000.0,500000.0,1000000.0,1,1,5/16,Монолитно-кирпичный,5,16,198.0,95.0,18.0
2,2,271173086,4,Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,500000.0,500000.0,1000000.0,1,0,5/16,Не указано,5,16,200.0,116.0,4.0


##### Заполнили пропуски в показателе **Высота потолков** с помощью среднего значения по этому столбцу

In [41]:
mean_ceiling = df['Ceiling_height_m'].mean()
df['Ceiling_height_m'].fillna(mean_ceiling, inplace=True)

/tmp/ipykernel_26000/2112350802.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Ceiling_height_m'].fillna(mean_ceiling, inplace=True)


##### Добавляем новые показатели Время от дома до метро - Time_metro, Станция метро - St_metro

In [42]:
df['Time_metro'] = df["Metro"].str.split('(').str[1]
df['St_metro'] = df["Metro"].str.split('(').str[0]
df['Time_metro'] = df['Time_metro'].str.extract('(\d+)').astype(float)
most_frequent = df['Time_metro'].mode() [0]
df['Time_metro'].fillna(most_frequent, inplace=True)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_26000/4133265629.py:3: SyntaxWarning: invalid escape sequence '\d'
  df['Time_metro'] = df['Time_metro'].str.extract('(\d+)').astype(float)
/tmp/ipykernel_26000/4133265629.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Time_metro'].fillna(most_frequent, inplace=True)


##### Заполнили пропуски в показателях **Ремонт**, **Окна**, **Балконы**, **Санузел**, в каждом пропущенном занчении указали строку "Нет данных"

In [43]:
missing_Renovation = df['Renovation'].isnull().sum()
df['Renovation'].fillna("нет данных", inplace=True)

missing_Windows = df['Windows'].isnull().sum()
df['Windows'].fillna("нет данных", inplace=True)

missing_Balcony = df['Balcony'].isnull().sum()
df['Balcony'].fillna("нет данных", inplace=True)

missing_Balcony = df['Bathroom'].isnull().sum()
df['Bathroom'].fillna("нет данных", inplace=True)

/tmp/ipykernel_26000/913927111.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Renovation'].fillna("нет данных", inplace=True)
/tmp/ipykernel_26000/913927111.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

##### В показателе **Ремонт** изменили тип данных, вычислили частоту повторяющихся категорий: Без ремонта, Косметический, Дизайниреский, Евроремонт и Нет данных

In [44]:
df_1 = df.groupby("Renovation")['Unnamed: 0'].count().reset_index()
df_1 = df_1.rename(columns={'Unnamed: 0': 'Freq'})
sum = df_1['Freq'].sum()
df_1['Freq'] = df_1['Freq'].div(sum)
arr = df_1.to_numpy()
for i in range(0, len(arr)):
    df.loc[df['Renovation'] == arr[i][0], 'Renovation'] = arr[i][1]
print(df['Renovation'])

0        0.140498
1        0.140498
2        0.354866
3        0.354866
4        0.354866
           ...   
23363    0.354866
23364    0.354866
23365    0.140498
23366    0.354866
23367    0.372954
Name: Renovation, Length: 19737, dtype: object


##### В показателе **Балконы** изменили тип данных, вычислили частоту повторяющихся категорий: Балконы, Лоджии с указанием количества и Нет данных

In [45]:
df_1 = df.groupby("Balcony")['Unnamed: 0'].count().reset_index()
df_1 = df_1.rename(columns={'Unnamed: 0': 'Freq'})
sum = df_1['Freq'].sum()
df_1['Freq'] = df_1['Freq'].div(sum)
arr = df_1.to_numpy()
for i in range(0, len(arr)):
    df.loc[df['Balcony'] == arr[i][0], 'Balcony'] = arr[i][1]
print(df['Balcony'])

0        0.335917
1        0.335917
2        0.335917
3        0.335917
4        0.335917
           ...   
23363    0.321021
23364    0.254953
23365    0.321021
23366    0.254953
23367    0.254953
Name: Balcony, Length: 19737, dtype: object


##### В показателе **Окна** изменили тип данных, вычислили частоту повторяющихся категорий: На улицу, на улицу и двор, во двор и Нет данных

In [46]:
df_1 = df.groupby("Windows")['Unnamed: 0'].count().reset_index()
df_1 = df_1.rename(columns={'Unnamed: 0': 'Freq'})
sum = df_1['Freq'].sum()
df_1['Freq'] = df_1['Freq'].div(sum)
arr = df_1.to_numpy()
for i in range(0, len(arr)):
    df.loc[df['Windows'] == arr[i][0], 'Windows'] = arr[i][1]
print(df['Windows'])

0        0.260931
1         0.13994
2         0.13994
3         0.13994
4         0.13994
           ...   
23363    0.260931
23364    0.491868
23365     0.13994
23366    0.491868
23367    0.491868
Name: Windows, Length: 19737, dtype: object


##### В показателе **Санузел** изменили тип данных, вычислили частоту повторяющихся категорий: Совмещенный, раздельный с указанием количества и Нет данных

In [47]:
df_1 = df.groupby("Bathroom")['Unnamed: 0'].count().reset_index()
df_1 = df_1.rename(columns={'Unnamed: 0': 'Freq'})
sum = df_1['Freq'].sum()
df_1['Freq'] = df_1['Freq'].div(sum)
arr = df_1.to_numpy()
for i in range(0, len(arr)):
    df.loc[df['Bathroom'] == arr[i][0], 'Bathroom'] = arr[i][1]
print(df['Bathroom'])

0         0.10341
1        0.009069
2        0.011299
3        0.011299
4        0.065511
           ...   
23363    0.430663
23364    0.430663
23365    0.430663
23366    0.036986
23367     0.31104
Name: Bathroom, Length: 19737, dtype: object


##### В показателе **Тип здания** изменили тип данных, вычислили частоту повторяющихся категорий: Корпичный, монолитный и Нет данных

In [48]:
df_1 = df.groupby("Type_of_building_of_the_house")['Unnamed: 0'].count().reset_index()
df_1 = df_1.rename(columns={'Unnamed: 0': 'Freq'})
sum = df_1['Freq'].sum()
df_1['Freq'] = df_1['Freq'].div(sum)
arr = df_1.to_numpy()
for i in range(0, len(arr)):
    df.loc[df['Type_of_building_of_the_house'] == arr[i][0], 'Type_of_building_of_the_house'] = arr[i][1]
print(df['Type_of_building_of_the_house'])

0        0.183159
1        0.044181
2        0.150529
3        0.150529
4          0.3384
           ...   
23363    0.150529
23364    0.183159
23365    0.187263
23366    0.183159
23367      0.3384
Name: Type_of_building_of_the_house, Length: 19737, dtype: object


##### Разделение колонки **Можно с детьми/животными** на две отдельные и их заполнение 0 и 1 в зависимости от условия

In [49]:
def split_allowances(df, source_column):
    df['Allow_Children'] = df[source_column].str.contains('Можно с детьми', na=False).astype(int)
    df['Allow_Animals'] = df[source_column].str.contains('Можно с животными', na=False).astype(int)
    return df

df = split_allowances(df, 'With_children/animals')

df['Allow_Children'].fillna(0, inplace=True)
df['Allow_Animals'].fillna(0, inplace=True)

/tmp/ipykernel_26000/195561266.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Allow_Children'].fillna(0, inplace=True)
/tmp/ipykernel_26000/195561266.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

##### Удаление колонок колонок, которые дублируют информацию

In [50]:
df = df.drop(['Metro', 'Phone', 'Description', 'Garbage_chute', 'Link_to_the_ad', 'Type', 'Additionally', 'Price', 'Area_m2', 'Home', 'Elevator', 'Room_area_m2', 'With_children/animals'], axis='columns')

In [51]:
df.isnull().sum()

Unnamed: 0                         0
ID                                 0
Number_of_rooms                    0
Address                            0
Renovation                         0
Balcony                            0
Windows                            0
Bathroom                           0
Ceiling_height_m                   0
Rent                               0
Deposit                            0
Result                             0
Passenger_elevator                 0
Freight_elevator                   0
Number_of_floors                   0
Type_of_building_of_the_house      0
Current_floor                      0
Total_number_of_floors             0
Total_area                         0
Living_area                        0
Kitchen                            0
Time_metro                         0
St_metro                         346
Allow_Children                     0
Allow_Animals                      0
dtype: int64

In [52]:
df.tail()



,Unnamed: 0,ID,Number_of_rooms,Address,Renovation,Balcony,Windows,Bathroom,Ceiling_height_m,Rent,Deposit,Result,Passenger_elevator,Freight_elevator,Number_of_floors,Type_of_building_of_the_house,Current_floor,Total_number_of_floors,Total_area,Living_area,Kitchen,Time_metro,St_metro,Allow_Children,Allow_Animals
23363,23363,215565511,2,"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",0.354866,0.321021,0.260931,0.430663,2.992925,42000.0,42000.0,84000.0,1,1,10/14,0.150529,10,14,35.0,16.4,8.0,8.0,м. Говорово,0,0
23364,23364,274654844,1,"Москва, Производственная улица, 8к1, ш. Боровс...",0.354866,0.254953,0.491868,0.430663,2.992925,45000.0,45000.0,90000.0,1,1,5/18,0.183159,5,18,38.7,16.5,11.0,7.0,м. Солнцево,0,0
23365,23365,268679909,2,"Москва, Боровский проезд, 11",0.140498,0.321021,0.13994,0.430663,2.992925,50000.0,50000.0,100000.0,0,0,5/5,0.187263,5,5,43.1,0.0,0.0,6.0,м. Солнцево,1,0
23366,23366,274807525,2,"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",0.354866,0.254953,0.491868,0.036986,2.650000,55000.0,50000.0,105000.0,3,0,8/23,0.183159,8,23,52.5,10.0,0.0,11.0,м. Солнцево,0,0
23367,23367,274672243,2,"Москва, улица Богданова, 2к1, ш. Боровское (3 ...",0.372954,0.254953,0.491868,0.31104,2.992925,57000.0,60000.0,117000.0,3,0,6/23,0.3384,6,23,90.0,48.2,15.5,10.0,м. Говорово,1,1


In [53]:
df = df.dropna(subset=['St_metro'])
df.isnull().sum()

Unnamed: 0                       0
ID                               0
Number_of_rooms                  0
Address                          0
Renovation                       0
Balcony                          0
Windows                          0
Bathroom                         0
Ceiling_height_m                 0
Rent                             0
Deposit                          0
Result                           0
Passenger_elevator               0
Freight_elevator                 0
Number_of_floors                 0
Type_of_building_of_the_house    0
Current_floor                    0
Total_number_of_floors           0
Total_area                       0
Living_area                      0
Kitchen                          0
Time_metro                       0
St_metro                         0
Allow_Children                   0
Allow_Animals                    0
dtype: int64

In [54]:
df.head()

,Unnamed: 0,ID,Number_of_rooms,Address,Renovation,Balcony,Windows,Bathroom,Ceiling_height_m,Rent,Deposit,Result,Passenger_elevator,Freight_elevator,Number_of_floors,Type_of_building_of_the_house,Current_floor,Total_number_of_floors,Total_area,Living_area,Kitchen,Time_metro,St_metro,Allow_Children,Allow_Animals
0,0,271271157,4,"Москва, улица Новый Арбат, 27",0.140498,0.335917,0.260931,0.10341,3.0,500000.0,500000.0,1000000.0,4,1,5/16,0.183159,5,16,200.0,20.0,0.0,9.0,м. Смоленская,1,1
1,1,271634126,4,"Москва, улица Новый Арбат, 27",0.140498,0.335917,0.13994,0.009069,3.5,500000.0,500000.0,1000000.0,1,1,5/16,0.044181,5,16,198.0,95.0,18.0,8.0,м. Смоленская,1,0
2,2,271173086,4,"Москва, улица Новый Арбат, 27",0.354866,0.335917,0.13994,0.011299,3.2,500000.0,500000.0,1000000.0,1,0,5/16,0.150529,5,16,200.0,116.0,4.0,7.0,м. Смоленская,1,0
3,3,272197456,4,"Москва, переулок Плотников, 21С1",0.354866,0.335917,0.13994,0.011299,3.2,400000.0,400000.0,800000.0,1,0,5/6,0.150529,5,6,170.0,95.0,17.0,3.0,м. Смоленская,0,1
4,4,273614615,2,"Москва, улица Новый Арбат, 15",0.354866,0.335917,0.13994,0.065511,3.9,225000.0,225000.0,450000.0,1,1,12/26,0.3384,12,26,58.0,38.0,5.0,7.0,м. Арбатская,0,0


In [59]:
df.drop(['Address', 'St_metro', 'Unnamed: 0'], axis='columns')
print(df)

       Unnamed: 0         ID  Number_of_rooms  \
0               0  271271157                4   
1               1  271634126                4   
2               2  271173086                4   
3               3  272197456                4   
4               4  273614615                2   
...           ...        ...              ...   
23363       23363  215565511                2   
23364       23364  274654844                1   
23365       23365  268679909                2   
23366       23366  274807525                2   
23367       23367  274672243                2   

                                                 Address Renovation   Balcony  \
0                          Москва, улица Новый Арбат, 27   0.140498  0.335917   
1                          Москва, улица Новый Арбат, 27   0.140498  0.335917   
2                          Москва, улица Новый Арбат, 27   0.354866  0.335917   
3                       Москва, переулок Плотников, 21С1   0.354866  0.335917   
4     

In [60]:
df = pd.DataFrame(df)
df.to_csv('aiogram_data.csv', index=False, encoding='utf-8-sig')